In [38]:
import pyspark
from pyspark.sql import Row
import pandas as pd
from datetime import datetime, date

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.appName("Practice").getOrCreate()
# https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf

In [2]:
df = spark.read.option('header', 'true').csv('nba_tweets.csv')

In [3]:
df_filter = df.filter(df.id == '1478885309626544130')
df_filter.head()

22/02/12 18:39:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Row(id='1478885309626544130', conversation_id='1478884437991608323', created_at='2022-01-05 18:25:08 CST', date='2022-01-05', time='18:25:08', timezone='-0600', user_id='1373081665916915714', username='pipehennessy', name='CJ Stroud Heisman', place=None, tweet='@LegendOfWinning Nigga fat talking about nba players', language='en', mentions='[]', urls='[]', photos='[]', replies_count='0', retweets_count='0', likes_count='0', hashtags='[]', cashtags='[]', link='https://twitter.com/PipeHennessy/status/1478885309626544130', retweet='False', quote_url=None, video='0', thumbnail=None, near=None, geo=None, source=None, user_rt_id=None, user_rt=None, retweet_id=None, reply_to="[{'screen_name': 'LegendOfWinning', 'name': 'LegendOfWinning', 'id': '1642136329'}]", retweet_date=None, translate=None, trans_src=None, trans_dest=None)

In [6]:
df.columns

['id',
 'conversation_id',
 'created_at',
 'date',
 'time',
 'timezone',
 'user_id',
 'username',
 'name',
 'place',
 'tweet',
 'language',
 'mentions',
 'urls',
 'photos',
 'replies_count',
 'retweets_count',
 'likes_count',
 'hashtags',
 'cashtags',
 'link',
 'retweet',
 'quote_url',
 'video',
 'thumbnail',
 'near',
 'geo',
 'source',
 'user_rt_id',
 'user_rt',
 'retweet_id',
 'reply_to',
 'retweet_date',
 'translate',
 'trans_src',
 'trans_dest']

In [11]:
geo_aggs = df.groupBy("geo").count().collect() # collect will pull it out of the pyspark gibberish and into a python native data structure.

In [39]:
df_geo = pd.DataFrame(df.na.fill('Uncollected', subset = ["geo"]).select("id", "created_at", "tweet", "geo").collect())
df_geo = pd.DataFrame(df.na.fill('Uncollected', subset = ["geo"]).collect())

In [35]:
df_exact_text = df[df.tweet.isin("@woledwar explain in nba terms", "orselectthis")].collect() # this is an exact str match.
df_text = df.select("id", df.tweet \
    .startswith("Desmond")) \
    .collect()
    # # only rows with demond will be true, all others false.

In [37]:
df.select("tweet", "id").show()

+----------------------------------+-------------------+
|                             tweet|                 id|
+----------------------------------+-------------------+
|              @LegendOfWinning ...|1478885309626544130|
|              Desmond Bane (qua...|1478885303704342528|
|               https://t.co/oEX...|1478885303695724546|
|               https://t.co/s3z...|1478885296217358338|
|              @PauladaJr Tinha ...|1478885294841683968|
|              "Beal and Christi...|1478885294824898562|
|              10.4.2019 - Last ...|1478885291750576145|
|【告知】私の発行するNBAメルマガ...|1478885290848964611|
|              NBA Leader: 27.4 ...|1478885289082818561|
|              @DirtyJerz32 @OSU...|1478885288848105474|
|              @E_Pistons_Pod At...|1478885286394437634|
|              Denver Nuggets vs...|1478885282992689153|
|              "nobody:  NBA com...|1478885277179555843|
|              From unknown to u...|1478885276931997701|
|              #ThunderUp has co...|147888527

In [50]:
# df_query = pd.DataFrame(df.select("tweet", "geo", F.when(df.geo == "en", 1).otherwise(0).alias("is_boolean_column")).collect())
df_query = df.select("tweet", "geo", F.when(df.geo == "en", 1).otherwise(0).alias("is_boolean_column")).show()

+----------------------------------+----+-----------------+
|                             tweet| geo|is_boolean_column|
+----------------------------------+----+-----------------+
|              @LegendOfWinning ...|null|                0|
|              Desmond Bane (qua...|null|                0|
|               https://t.co/oEX...|null|                0|
|               https://t.co/s3z...|null|                0|
|              @PauladaJr Tinha ...|null|                0|
|              "Beal and Christi...|null|                0|
|              10.4.2019 - Last ...|null|                0|
|【告知】私の発行するNBAメルマガ...|null|                0|
|              NBA Leader: 27.4 ...|null|                0|
|              @DirtyJerz32 @OSU...|null|                0|
|              @E_Pistons_Pod At...|null|                0|
|              Denver Nuggets vs...|null|                0|
|              "nobody:  NBA com...|null|                0|
|              From unknown to u...|null|             

In [65]:
# renmame id to tweet_id, and remove the conversation_id column
df2 = df.withColumnRenamed("id", "tweet_id") \
    .drop("conversation_id")
df2.write \
    .mode('overwrite') \
    .save('newparquet.parquet')

df3 = spark.read.load('newparquet.parquet')
df4_list = df3.collect()
df4_df = pd.DataFrame(df4_list)
df4_new = df3.toPandas()

In [77]:
bb1 = spark.read.csv('nba_tweets.csv', header = True)
print(f"the count before filter is {bb1.count()}")
from pyspark.sql.functions import column as col
bb2 = bb1.where(col('geo') == 'en')
print(f"the counter after filter is {bb2.count()}")

the count before filter is 22598
the counter after filter is 5


In [54]:
df2.printSchema()

root
 |-- tweet_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- timezone: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- username: string (nullable = true)
 |-- name: string (nullable = true)
 |-- place: string (nullable = true)
 |-- tweet: string (nullable = true)
 |-- language: string (nullable = true)
 |-- mentions: string (nullable = true)
 |-- urls: string (nullable = true)
 |-- photos: string (nullable = true)
 |-- replies_count: string (nullable = true)
 |-- retweets_count: string (nullable = true)
 |-- likes_count: string (nullable = true)
 |-- hashtags: string (nullable = true)
 |-- cashtags: string (nullable = true)
 |-- link: string (nullable = true)
 |-- retweet: string (nullable = true)
 |-- quote_url: string (nullable = true)
 |-- video: string (nullable = true)
 |-- thumbnail: string (nullable = true)
 |-- near: string (nullable = true)
 |-- geo: stri